# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [112]:
%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.sum(l_c * l_c / l_s + r_c * r_c / r_s, axis=1))
        / ((l_s + r_s).T[0])  # Ваш код в 1 строчку

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum(l_c * np.log2(l_c / l_s) +
                        r_c * np.log2(r_c / r_s), axis=1)) / ((l_s + r_s).T[0])
        # Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(r_c, axis=1) + np.max(l_c, axis=1))
        / ((r_s + l_s).T[0])  # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.sqrt(n_feature))])
        # Ваш код в 1 строчку

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.log2(n_feature))])
        # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)  # Ваш код в 1 строчку

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        '''
        Сортирует х и y по x, cчитает мин.
        кол-во эл-тов для поддеревьев
        '''
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        '''
        Обрубает определённую часть массива,
        ищет эл-ты, у которых смена класса
        '''
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        '''
        Кол-во равных эл-тов. Единицы - места смены
        Матрица числа классов у каждого из интервалов
        '''
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        '''
        Подсчёт кол-ва эл-тов в каждом классе по обе стороны раздела
        и кол-ва эл-тов там и там
        '''
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        '''
        Ищет аргумент, при котором мин. неопр-ть
        '''
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        '''
        Возвр. вел-ну неопр-ти и место раздела
        '''
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if pred_f == -1:
            self.feature_importances_ = np.zeros(x.shape[1])
            self.X_size = x.shape[0]
        cl = (np.bincount(y, minlength=self.num_class)) / x.shape[0]
        y_cl = np.argmax(cl)
        if self.max_depth is None:
            self.max_depth = np.inf
        if ((depth > self.max_depth) or
            (cl[y_cl] >= self.sufficient_share) or
                (x.shape[0] <= self.min_samples_split)):
            self.tree[node_id] = (self.LEAF_TYPE, y_cl, cl)
        else:
            ids = self.get_feature_ids(x.shape[1])
            a = np.inf
            ind = -1
            for i in ids:
                inp, tsh = self.__find_threshold(x[:, i], y)
                if inp < a:
                    ind = i
                    thr = tsh
                    a = inp
            if(ind == -1):
                self.tree[node_id] = (self.LEAF_TYPE, y_cl, cl)
            else:
                x_left, x_right, y_left, y_right = \
                 self.__div_samples(x, y, ind, thr)
                if x_left.size == 0 or x_right.size == 0:
                    self.tree[node_id] = (self.LEAF_TYPE, y_cl, cl)
                else:
                    if self.G_function == self.__gini:
                        fc = 1 - np.sum(cl ** 2)
                    elif self.G_function == self.__entropy:
                        fc = -np.sum(np.log2(cl) * cl)
                    else:
                        fc = 1 - np.max(cl)
                    self.feature_importances_[ind] += \
                        x.shape[0] * (fc - a) / self.X_size
                    self.tree[node_id] = (self.NON_LEAF_TYPE, ind,
                                          thr)
                    self.__fit_node(x_left, y_left, node_id * 2 + 1,
                                    depth + 1, 1)
                    self.__fit_node(x_right, y_right, node_id * 2 + 2,
                                    depth + 1, 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [113]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [114]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [146]:
%time clf.fit(X_train, y_train)

Wall time: 160 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [147]:
%time my_clf.fit(X_train, y_train)

Wall time: 1.58 s


## Проверка качества работы на wine

In [148]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5428264084980503

In [149]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.46771037181996084

## Подготовка данных Speed Dating Data 

In [131]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb',
              'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o','met_o', 'field', 'undergra', 'from', 'zipcode', 'career', 'sports', 'tvsports',
              'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'expnum', 'wave'],
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
f = ['attr{}_1'.format(4), 'sinc{}_1'.format(4), 'intel{}_1'.format(4), 'fun{}_1'.format(4), 'amb{}_1'.format(4), 'shar{}_1'.format(4)]
df = df.drop(f, axis=1)
f = ['attr{}_1'.format(5), 'sinc{}_1'.format(5), 'intel{}_1'.format(5), 'fun{}_1'.format(5), 'amb{}_1'.format(5), 'shar{}_1'.format(5)]
f.remove('shar{}_1'.format(5))
df = df.drop(f, axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']).drop(['gender'], axis=1).dropna()
df_fem = df.query('gender == 0').drop_duplicates(subset=['iid']).drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()
df_fem.columns = df_fem.columns + '_f'
df_fem = df_fem.drop(['pid_f'], axis=1)
df_df = df_male.join(df_fem.set_index('iid_f'), on='pid', how='inner')
df_df = df_df.drop(['iid', 'pid'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_df.iloc[:, 1:].values, df_df.iloc[:, 0].values, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [140]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

Wall time: 172 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [141]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

Wall time: 1.79 s


## Проверка качества работы на Speed Dating Data

In [142]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5428264084980503

In [143]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.46771037181996084

## Задание 3

In [144]:
print(pd.Series(data=clf.feature_importances_, index=df_pair.columns[1:]).sort_values(ascending=False).head(10))
print(pd.Series(data=my_clf.feature_importances_, index=df_pair.columns[1:]).sort_values(ascending=False).head(10))

int_corr      0.087954
sinc1_1       0.033887
age_f         0.031027
fun2_1_f      0.030226
field_cd_f    0.029241
intel2_1_f    0.028670
amb2_1_f      0.027282
imprelig_f    0.023724
attr1_1       0.023625
fun3_1        0.023028
dtype: float64
amb1_1_f      0.006596
date          0.006562
income        0.005741
int_corr      0.004770
go_out_f      0.004476
fun1_1        0.004002
field_cd_f    0.003755
go_out        0.003177
intel2_1      0.003152
income_f      0.003057
dtype: float64


## Задание 4

In [145]:
p = {
    'criterion': ['entropy', 'gini'],
    'max_depth': range(1, 30),
    'min_samples_leaf': range(1, 20),
    'n_estimators': range(1, 150)
}
rscv = RandomizedSearchCV(RandomForestClassifier(),
                          param_distributions=p)
rscv.fit(df_df.iloc[:, 1:].values, df_df.iloc[:, 0].values)
rscv.best_params_

{'n_estimators': 59,
 'min_samples_leaf': 13,
 'max_depth': 1,
 'criterion': 'gini'}